In [21]:
from urllib.request import urlopen

In [22]:
from bs4 import BeautifulSoup

In [23]:
import urllib.request, urllib.parse, urllib.error

In [24]:
from pdfminer.pdfparser import PDFParser

In [25]:
from pdfminer.pdfdocument import PDFDocument

In [26]:
import sys

In [27]:
import cchardet

In [1]:
import csv

In [2]:
import pandas as pd

In [6]:
# Lista de links com os PDFs

In [30]:
emendas = urlopen("http://www.camara.gov.br/proposicoesWeb/prop_emendas?idProposicao=2122076&subst=0")

In [31]:
bsObje =  BeautifulSoup(emendas, "lxml")

In [32]:
pdfs = bsObje.findAll("a", {"class":"rightIconified iconDetalhe linkDownloadTeor"})

In [33]:
# Lista dos nomes de deputados

In [34]:
req = urlopen('http://www.camara.gov.br/proposicoesWeb/prop_emendas?idProposicao=2122076&subst=0')
soup = BeautifulSoup(req.read(), 'html.parser')

tables_ele = soup.findAll('tbody', {'class': 'coresAlternadas'})
deputados = []
for table_ele in tables_ele:
    for row in table_ele.findAll('tr'):
        cols = row.findAll('td')
        deputados.append(cols[3].text.strip())

conta = 0

In [35]:
# Cria repositório final e cabeçalho

In [36]:
tabela_final = []

In [37]:
tabela_final.append(["Arquivo", "Deputado", "Autor", "Assunto", "Data_Criacao", "Data_Moficacao"])

In [38]:
# Função para testar codecs
def str_decode(str):
    if str == None:
        return str
    else: 
        # Verifica qual o codec do sistema atual (codec "padrão")
        defaultcodec = sys.getdefaultencoding().lower()

        codec = cchardet.detect(str)['encoding']

        if (defaultcodec != codec.lower()):
            return str.decode(codec) # Se o codec for diferente do sistema atual então decodifica
        else:
            return str # Se o codec for o do sistema atual então mantêm 

# Iteração com os links de PDFs
for name in pdfs:
    # Cria url
    url = name.get('href')
    url = "http://www.camara.gov.br/proposicoesWeb/" + url
    
    # Limpeza para acesso
    posicao = url.find('filename=') + 9
    arquivo = (url[posicao:])
    arquivo = arquivo.replace('+', ' ')
    arquivo = arquivo.replace('/', '-')
    arquivo = arquivo.replace('%3D%3E', '=-')
    arquivo = arquivo + ".pdf"
    
    # Baixa o arquivo
    urllib.request.urlretrieve(url, arquivo)
    
    # Extrai metadados
    fp = open(arquivo, 'rb')
    parser = PDFParser(fp)
    doc = PDFDocument(parser)
    dados_recuperados = doc.info[0]
    
    # Testa e grava campos 
    author = str_decode(dados_recuperados.get("Author"))
    subject = str_decode(dados_recuperados.get("Subject"))
    creation_date = str_decode(dados_recuperados.get("CreationDate"))
    mod_date = str_decode(dados_recuperados.get("ModDate"))
    nome_deputado = deputados[conta]
    conta = conta + 1
    
    # Grava uma linha completa
    tabela_final.append([arquivo, nome_deputado, author, subject, creation_date, mod_date])

In [39]:
# Grava o CSV
header = True
# Abre um arquivo no computador e o prepara para gravação
with open('emendas_autores.csv', 'a') as f:
    # usa a ferramenta "writer" do módulo csv
    writer = csv.writer(f)
    # se precisarmos gravar o cabeçalho...
    if header is True:
        # gravamos o cabeçalho
        writer.writerow(tabela_final[0])
        # marcamos que não é mais preciso gravar o cabeçalho
        header = False
    # gravamos o restante das linhas no CSV
    writer.writerows(tabela_final[1:])

In [48]:
# Lê o arquivo CSV

In [3]:
emd = pd.read_csv("emendas_autores.csv", sep = ',', encoding = 'latin_1')

In [42]:
emd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1340 entries, 0 to 1339
Data columns (total 6 columns):
Arquivo           1340 non-null object
Deputado          1340 non-null object
Autor             1282 non-null object
Assunto           220 non-null object
Data_Criacao      1340 non-null object
Data_Moficacao    1307 non-null object
dtypes: object(6)
memory usage: 31.4+ KB


In [45]:
### Mostra as linhas finais e iniciais para teste

In [46]:
emd.tail().reset_index()

,index,Arquivo,Deputado,Autor,Assunto,Data_Criacao,Data_Moficacao
0,1335,EMP 28-2017 =- PL 6787-2016.pdf,Bebeto,NaN,NaN,D:20000101100657-03'00',NaN
1,1336,EMP 29-2017 =- PL 6787-2016.pdf,Bebeto,NaN,NaN,D:20000101100710-03'00',NaN
2,1337,EMP 30-2017 =- PL 6787-2016.pdf,Paulo Pereira da Silva,NaN,NaN,D:20000101083124-03'00',NaN
3,1338,EMP 31-2017 =- PL 6787-2016.pdf,Renata Abreu,NaN,NaN,D:20000101090357-03'00',NaN
4,1339,EMP 32-2017 =- PL 6787-2016.pdf,Evandro Gussi,NaN,NaN,D:20000101092046-03'00',NaN


In [47]:
emd.head().reset_index()

,index,Arquivo,Deputado,Autor,Assunto,Data_Criacao,Data_Moficacao
0,0,EMC 1-2017 PL678716 =- PL 6787-2016.pdf,Laura Carneiro,Ivanete de Araujo Costa,EMD ADI - Emenda Aditiva,D:20170314114321-07'00',D:20170314114321-07'00'
1,1,EMC 2-2017 PL678716 =- PL 6787-2016.pdf,Laura Carneiro,Ivanete de Araujo Costa,EMD ADI - Emenda Aditiva,D:20170314114328-07'00',D:20170314114328-07'00'
2,2,EMC 3-2017 PL678716 =- PL 6787-2016.pdf,Laura Carneiro,Ivanete de Araujo Costa,EMD ADI - Emenda Aditiva,D:20170314114336-07'00',D:20170314114336-07'00'
3,3,EMC 4-2017 PL678716 =- PL 6787-2016.pdf,Laura Carneiro,MOD.CONLE.ST 2130/2017 - P_6736 - Davi Ribeiro...,EMD ADI - Emenda Aditiva,D:20170314114344-07'00',D:20170314114344-07'00'
4,4,EMC 5-2017 PL678716 =- PL 6787-2016.pdf,Laura Carneiro,Ivanete de Araujo Costa,EMD ADI - Emenda Aditiva,D:20170314114352-07'00',D:20170314114352-07'00'


## Algumas análises

### A ordem dos autores de emendas em número de redigidas (obs. em algumas esse campo foi deixado em branco)

In [4]:
emd.Autor.value_counts().reset_index()

,index,Autor
0,P_4189,99
1,Prodasen,98
2,Ricardo de Saboya Rocha Miranda,88
3,Ivanete de Araujo Costa,61
4,Volia Cassar,50
5,Dep. Jeronimo Goergen,47
6,MOD.CONLE.ST 2593/2017 - P_5675 - Beatriz Reze...,46
7,Amanda Galvão Ferreira Tabosa,45
8,wlamir,38
9,Angelo Fabiano Farias da Costa,35


### Filtros pelos cinco primeiros principais autores, com respectivos deputados

In [19]:
top_autores = emd.Autor.value_counts().reset_index().head(5)

In [20]:
top_autores

,index,Autor
0,P_4189,99
1,Prodasen,98
2,Ricardo de Saboya Rocha Miranda,88
3,Ivanete de Araujo Costa,61
4,Volia Cassar,50


In [23]:
primeiro = emd[emd.Autor == "P_4189"]

In [25]:
primeiro.Deputado.value_counts().reset_index()

,index,Deputado
0,Mauro Lopes,22
1,Vanderlei Macris,20
2,Magda Mofatto,18
3,Gorete Pereira,11
4,Célio Silveira,10
5,Valdir Colatto,9
6,Arnaldo Jordy,8
7,Celso Maldaner,1


In [26]:
segundo = emd[emd.Autor == "Prodasen"]

In [27]:
segundo.Deputado.value_counts().reset_index()

,index,Deputado
0,Chico Alencar,23
1,Sergio Vidigal,21
2,Arnaldo Jordy,19
3,Hissa Abrahão,9
4,Daniel Almeida,9
5,André Figueiredo,7
6,Chico Lopes,3
7,Orlando Silva,3
8,Jô Moraes,3
9,Luiza Erundina,1


In [28]:
terceiro = emd[emd.Autor == "Ricardo de Saboya Rocha Miranda"]

In [29]:
terceiro.Deputado.value_counts().reset_index()

,index,Deputado
0,Major Olimpio,26
1,Diego Andrade,22
2,Renzo Braz,19
3,Valdir Colatto,11
4,Lelo Coimbra,10


In [30]:
quarto = emd[emd.Autor == "Ivanete de Araujo Costa"]

In [31]:
quarto.Deputado.value_counts().reset_index()

,index,Deputado
0,Benedita da Silva,12
1,Carlos Zarattini,12
2,Roberto Sales,11
3,Laura Carneiro,10
4,Celso Maldaner,8
5,Valdir Colatto,2
6,Marinaldo Rosendo,2
7,Diego Garcia,1
8,Herculano Passos,1
9,Waldenor Pereira,1


In [32]:
quinto = emd[emd.Autor == "Volia Cassar"]

In [33]:
quinto.Deputado.value_counts().reset_index()

,index,Deputado
0,Laura Carneiro,37
1,Wadih Damous,13
